In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
import sympy as sy
sy.init_printing() 

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# <font face="gotham" color="purple"> LU 分解</font>

LU 分解的出现是因为它的计算效率，它主要简化了解线性方程组的过程，尽管 LU 的浮点运算次数仍然高于高斯-约当消元。然而，如果你要计算多个 $A x =b$ 的解，LU 分解就特别方便了。

例如，如果你有一组 $\{b_i,\ i \in \mathbb Z\}$ 来代入系统，使得
$$
Ax=b_1,\quad Ax=b_2,\quad Ax=b_3,\quad Ax=b_4, \quad...
$$
我们只需要对 $A$ 进行一次分解，但是高斯-约当算法必须对每个 $b_i$ 重新进行操作。

## <font face="gotham" color="purple"> LU 算法</font>

我们的目标是将矩阵 $A$ 分解为 $L$ 和 $U$，分别表示_下三角矩阵_和_上三角矩阵_。而且 $L$ 必须在其主对角线上有 $1$。
$$
A = LU
$$
例如，
$$
A=\underbrace{\left[\begin{array}{cccc}
1 & 0 & 0 & 0 \\
* & 1 & 0 & 0 \\
* & * & 1 & 0 \\
* & * & * & 1
\end{array}\right]}_{L}\underbrace{\left[\begin{array}{ccccc}
\blacksquare & * & * & * & * \\
0 & \blacksquare & * & * & * \\
0 & 0 & 0 & \blacksquare & * \\
0 & 0 & 0 & 0 & 0
\end{array}\right]}_{U}
$$

分解的可行性取决于是否可以通过一系列行操作将 $A$ 转换为上三角矩阵，即

$$E_p...E_2E_1  A = U$$

然后

$$A = (E_p...E_2E_1)^{-1}U = LU$$

其中

$$L = (E_p...E_2E_1)^{-1}$$

我们将在这里手动计算一个示例：

$$
A =
\begin{bmatrix}
9 & 3 & 6\\3 & 4 & 6\\0 & 8 & 8\\
\end{bmatrix}
$$

$$
\underbrace{
\begin{bmatrix}
1 & 0 & 0\\-\frac{1}{3} & 1 & 0\\0 & 0 & 1\\
\end{bmatrix}}_{E_1}
\underbrace{
\begin{bmatrix}
9 & 3 & 6\\3 & 4 & 6\\0 & 8 & 8\\ 
\end{bmatrix}}_{A}
=
\underbrace{
\begin{bmatrix}
9 & 3 & 6\\0 & 3 & 4\\0 & 8 & 8\\ 
\end{bmatrix}}_{E_1A}
$$

$$
\underbrace{
\begin{bmatrix}
1 & 0 & 0\\0 &1  & 0\\0 & -\frac{8}{3} & 1\\
\end{bmatrix}}_{E_2}
\underbrace{
\begin{bmatrix}
9 & 3 & 6\\0 & 3 & 4\\0 & 8 & 8\\ 
\end{bmatrix}}_{E_1A}
=
\underbrace{
\begin{bmatrix}
9 & 3 & 6\\0 & 3 & 4\\0 & 0 & -\frac{8}{3}\\ 
\end{bmatrix}}_{E_2E_1A=U}
$$

$$
L^{-1} = E_2E_1 = 
\underbrace{
\begin{bmatrix}
1 & 0 & 0\\0 &1  & 0\\0 & -\frac{8}{3} & 1\\
\end{bmatrix}}_{E_2}
\underbrace{
\begin{bmatrix}
1 & 0 & 0\\-\frac{1}{3} & 1 & 0\\0 & 0 & 1\\
\end{bmatrix}}_{E_1}
=
\begin{bmatrix}
1 & 0 & 0\\-\frac{1}{3} & 1 & 0\\\frac{8}{9} & -\frac{8}{3} & 1\\
\end{bmatrix}
$$

或者我们可以直接计算 $E_1^{-1}$ 和 $E_2^{-1}$，因为
$$
L = E_1^{-1}E_2^{-1}
$$


构造 $E_2$ 和 $E_1$ 的增广矩阵
$$
[E_1| I]=\left[\begin{array}{ccc|ccc}  
1 & 0 & 0 & 1 & 0 & 0\\
-\frac{1}{3} & 1 & 0 & 0  & 1 & 0\\
0 & 0 & 1 & 0 & 0 & 1
\end{array}\right]
\sim
\left[\begin{array}{ccc|ccc}
1 &0 &0 &1 & 0 & 0\\
0& 1& 0& \frac{1}{3} & 1 & 0\\
0 & 0 & 1 &0 & 0 & 1
\end{array}\right]
=[I|E_1^{-1}]
$$

$$
[E_2| I]=
\left[\begin{array}{ccc|ccc}  
1 & 0 & 0 & 1 & 0 & 0\\
0 & 1 & 0 & 0  & 1 & 0\\
0 & -\frac{8}{3} & 1 & 0 & 0 & 1\\
\end{array}
\right]
\sim
\left[\begin{array}{ccc|ccc}  
1 & 0 & 0 & 1 & 0 & 0\\
0 & 1 & 0 & 0  & 1 & 0\\
0 & 0 & 1 & 0 &\frac{8}{3}& 1\\
\end{array}
\right]
=[I|E_2^{-1}]
$$

然后我们计算 $E_1$ 和 $E_2$ 的逆：
$$
E_1^{-1} = \left[\begin{array}{ccc|ccc} 1& 0& 0 \\ \frac{1}{3}& 1& 0\\ 0& 0 &1  \end{array}\right]\\
E_2^{-1} = \left[\begin{array}{ccc|ccc} 1& 0& 0 \\ 1& 1& 0\\ 0& \frac{8}{3} &1  \end{array}\right]
$$

$$
L = E_1^{-1}E_2^{-1}= 
\left[\begin{array}{ccc|ccc} 1& 0& 0 \\ \frac{1}{3}& 1& 0\\ 0& 0 &1  \end{array}\right]
\left[\begin{array}{ccc|ccc} 1& 0& 0 \\ 1& 1& 0\\ 0& \frac{8}{3} &1 \end{array}\right]
=
\left[\begin{array}{ccc|ccc} 1& 0& 0 \\ \frac{4}{3}&1& 0\\ 0& \frac{8}{3} &1 \end{array}\right]
$$

LU 分解的最终结果：
$$
A = LU =
\underbrace{
\left[\begin{array}{ccc|ccc} 1& 0& 0 \\ \frac{4}{3}&1& 0\\ 0& \frac{8}{3} &1 \end{array}\right]}_{L}
\underbrace{
\begin{bmatrix}
9 & 3 & 6\\0 & 3 & 4\\0 & 0 & -\frac{8}{3}\\ 
\end{bmatrix}}_{E_2E_1A=U}
$$

我们可以看一下 SciPy 的结果。

In [3]:
A = np.array([[9, 3, 6], [3, 4, 6], [0, 8, 8]]); A

array([[9, 3, 6],
       [3, 4, 6],
       [0, 8, 8]])

In [4]:
P, L, U = sp.linalg.lu(A)
P
L
U

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

array([[1.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.33333333, 0.375     , 1.        ]])

array([[9., 3., 6.],
       [0., 8., 8.],
       [0., 0., 1.]])

In [5]:
P@L@U # 这是 A

array([[9., 3., 6.],
       [3., 4., 6.],
       [0., 8., 8.]])

很容易注意到 SciPy 的 `lu` 函数不仅给出了 $L$ 和 $U$，还给出了 $P$，这是一个**置换矩阵**。理论上，我们不需要行交换来将 $A$ 转换为 $U$，但在某些情况下，我们必须提前进行行交换，否则分解将无法实现。

因此，Scipy 使用 $PLU$ 分解来确保过程始终可行
$$
A  = PLU
$$

实际上，$P = P^{-1}$，为什么呢？在增广矩阵表达式中分析起来更容易，行交换的初等矩阵的逆矩阵就是它们自己。

$$
[P| I]=\left[\begin{array}{ccc|ccc}  
1 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 1 & 0 & 1 & 0\\
0 & 1 & 0 & 0 & 0 & 1
\end{array}\right]
\sim
\left[\begin{array}{ccc|ccc}  
1 & 0 & 0 & 1 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 1\\
0 & 0 & 1 & 0 & 1 & 0
\end{array}\right]=[I| P^{-1}]  =[P^{-1}|I]
$$

有了这些知识，我们实际上正在分解的是

$$
PA = LU
$$

$$
\begin{align}
E_2E_1E_0A &=
\underbrace{
\begin{bmatrix}
1 & 0& 0\\0 & 1 & 0\\0 & -\frac{3}{8} & 1\\
\end{bmatrix}}_{E_2}
\underbrace{
\begin{bmatrix}
1 & 0& 0\\0 & 1 & 0\\-\frac{1}{3} & 0 & 1\\
\end{bmatrix}}_{E_1}
\underbrace{
\begin{bmatrix}
1 & 0& 0\\0 & 0 & 1\\0 & 1 & 0\\
\end{bmatrix}}_{E_0}
\begin{bmatrix}
9 & 3 & 6\\3 & 4 & 6\\0 & 8 & 8\\
\end{bmatrix}\\
E_2E_1E_0A &=
\underbrace{
\begin{bmatrix}
1 & 0& 0\\0 & 1 & 0\\0 & -\frac{3}{8} & 1\\
\end{bmatrix}}_{E_2}
\underbrace{
\begin{bmatrix}
1 & 0& 0\\0 & 1 & 0\\-\frac{1}{3} & 0 & 1\\
\end{bmatrix}}_{E_1}
\underbrace{
\begin{bmatrix}
9& 3& 6\\0&8 &8\\ 3& 4& 6
\end{bmatrix}}_{E_0A}\\
E_2E_1E_0A &=
\underbrace{
\begin{bmatrix}
1 & 0& 0\\0 & 1 & 0\\0 & -\frac{3}{8} & 1\\
\end{bmatrix}}_{E_2}
\underbrace{
\begin{bmatrix}
9 &3 &6\\0 &8& 8 \\0& 3& 4
\end{bmatrix}}_{E_1E_0A}\\
E_2E_1E_0A &=
\underbrace{
\begin{bmatrix}
9 &3 &6\\0 &8& 8 \\0& 0& -1
\end{bmatrix}}_{E_2E_1E_0A}=U
\end{align}
$$
重新排列一下，我们可以看到 $PL$：
$$
A = \underbrace{E_0^{-1}}_{P} \underbrace{(E_1^{-1}E_2^{-1})}_{L}U
$$

# <font face="gotham" color="purple"> 通过使用 LU 分解求解线性系统</font>

求解线性系统：
$$
\begin{align}
3x_1-7x_2 -2x_3+2x_4&=-9\\
-3x_1+5x_2 +x_3 &=5\\
6x_1-4x_2 -5x_4&=7\\
-9x_1+5x_2 -5x_3+12x_4&=11\\
\end{align}
$$
以矩阵形式：
$$
\underbrace{\left[\begin{array}{rrrr}
3 & -7 & -2 & 2 \\
-3 & 5 & 1 & 0 \\
6 & -4 & 0 & -5 \\
-9 & 5 & -5 & 12
\end{array}\right]}_{A}
\left[\begin{array}{r}
x_1 \\
x_2 \\
x_3 \\
x_4
\end{array}\right]
=
\underbrace{\left[\begin{array}{r}
-9 \\
5 \\
7 \\
11
\end{array}\right]}_{b}
$$
进行 $LU$ 分解：

$$\underbrace{\left[\begin{array}{rrrr}
3 & -7 & -2 & 2 \\
-3 & 5 & 1 & 0 \\
6 & -4 & 0 & -5 \\
-9 & 5 & -5 & 12
\end{array}\right]}_{A}
=\underbrace{
\left[\begin{array}{rrrr}
1 & 0 & 0 & 0 \\
-1 & 1 & 0 & 0 \\
2 & -5 & 1 & 0 \\
-3 & 8 & 3 & 1
\end{array}\right]}_{L}\underbrace{\left[\begin{array}{rrrr}
3 & -7 & -2 & 2 \\
0 & -2 & -1 & 2 \\
0 & 0 & -1 & 1 \\
0 & 0 & 0 & -1
\end{array}\right]}_{U}$$

将 $A$ 替换为 $LU$，我们得到 $L(Ux) = b$，现在解这对方程组

$$
Ly = b\\
Ux = y
$$

构造增广矩阵 $[L|b]$


$$
\underbrace{
\left[\begin{array}{rrrr}
1 & 0 & 0 & 0 \\
-1 & 1 & 0 & 0 \\
2 & -5 & 1 & 0 \\
-3 & 8 & 3 & 1
\end{array}\right]}_{L}
\underbrace{\left[\begin{array}{r}
y_1 \\
y_2 \\
y_3 \\
y_4
\end{array}\right]}_{y}
=
\underbrace{\left[\begin{array}{r}
-9 \\
5 \\
7 \\
11
\end{array}\right]}_{b}
$$

$$
\left[\begin{array}{rrrr|r}
1 & 0 & 0 & 0 & -9 \\
-1 & 1 & 0 & 0 & 5\\
2 & -5 & 1 & 0 & 7\\
-3 & 8 & 3 & 1 & 11
\end{array}\right]\sim
\left[\begin{array}{rrrr|r}
1 & 0 & 0 & 0 & -9 \\
0 & 1 & 0 & 0 & -4\\
0 & 0 & 1 & 0 & 5\\
0 & 0 & 0 & 1 & 1
\end{array}\right]
$$

$$
\left[\begin{array}{r}
y_1 \\
y_2 \\
y_3 \\
y_4
\end{array}\right]=
\left[\begin{array}{r}
-9 \\
-4 \\
5 \\
1
\end{array}\right]
$$

接下来我们解 $Ux = y$，在 NumPy 中展示如下：

In [6]:
U = np.array([[3, -7, -2, 2], [0, -2, -1, 2], [0, 0, -1, 1], [0, 0, 0,-1]])
y = np.array([-9, -4, 5, 1])
np.linalg.solve(U, y)

array([ 3.,  4., -6., -1.])

如果过程正确，这就是系统的解，我们可以通过调用 ```np.linalg.solve()``` 来验证结果。

In [7]:
A = np.array([[3, -7, -2, 2], [-3, 5, 1, 0], [6, -4, 0, -5], [-9, 5, -5, 12]])
b = np.array([-9, 5, 7, 11])
np.linalg.solve(A, b)

array([ 3.,  4., -6., -1.])

结果是一致的，$LU$ 分解有效！